# FORMULA 1 RACE DATA


El caso Kaggle del cual dispongo se denomina FORMULA 1 RACE DATA. En el se encuentra información de la FORMULA  desde la temporada 1950 hasta la temporada 2017. Este Kaggle esta formado por varios datasets, en los cuales se puede encontrar datos de cada piloto, escuderia, resultados, etc. Aunque el caso Kaggle contenga exactamente 13 datasets, no todos seran utlitzados debido a la gran cantidad de datos que resultaria.

Nada más ver el dataset que se me proporciono en lo primero que pensé es en intentar predecir algo relacionado con el piloto español más famoso de todos los tiempos en nuestro país, Fernando Alonso. Por lo que el trabajo girara entorno a el y en investigar y visualizar sus estadísticas a lo largo de todos estos años en la FORMULA 1.

Primeramente añadiremos todas las librerias necesarias para realizar la práctica y cargaremos los datasets con más importancia para nuestra investigación los cuales son: FALTA ESTO

In [26]:
# afegim llibreries necessaries
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import scipy.stats
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
import sklearn
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from statistics import mean
from sklearn import svm 
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import warnings 
warnings.filterwarnings('ignore')

#visualitzarem només 3 decimals per mostra
pd.set_option('display.float_format', lambda x: '%.3f' % x)

#llegim les dades del csv
def load_dataset(path):
    dataset = pd.read_csv(path, header=0, delimiter=',')
    return dataset

#carreguem dataset 
dataset = load_dataset('datasets/results.csv')
data = dataset.values

#carreguem dataset 
dataset2 = load_dataset('datasets/races.csv')
data2 = dataset2.values

#carreguem dataset 
#dataset3 = load_dataset('datasets/drivers.csv')
#data3 = dataset3.values

Como en este caso, a diferencia de prácticas anteriores, tenemos varios datasets. Realizaremos un join para asi unir todos ellos y tener todos los datos en un solo dataset.

In [27]:
inner_join_df = pd.merge(dataset, dataset2, on='raceId' , how='inner')
#inner_join_df2 = pd.merge(inner_join_df, dataset3, on='raceId' , how='inner')


NameError: name 'dataset3' is not defined

In [29]:
inner_join_df

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,...,fastestLapTime,fastestLapSpeed,statusId,year,round,circuitId,name,date,time_y,url
0,1,18,1,1,22.000,1,1.000,1,1,10.000,...,01:27.5,218.3,1,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_G...
1,2,18,2,2,3.000,5,2.000,2,2,8.000,...,01:27.7,217.586,1,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_G...
2,3,18,3,3,7.000,7,3.000,3,3,6.000,...,01:28.1,216.719,1,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_G...
3,4,18,4,4,5.000,11,4.000,4,4,5.000,...,01:28.6,215.464,1,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_G...
4,5,18,5,1,23.000,3,5.000,5,5,4.000,...,01:27.4,218.385,1,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_G...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23772,23777,988,842,5,10.000,17,16.000,16,16,0.000,...,01:43.8,192.542,11,2017,20,24,Abu Dhabi Grand Prix,2017-11-26,13:00:00,https://en.wikipedia.org/wiki/2017_Abu_Dhabi_G...
23773,23778,988,828,15,9.000,19,17.000,17,17,0.000,...,01:43.6,193.057,11,2017,20,24,Abu Dhabi Grand Prix,2017-11-26,13:00:00,https://en.wikipedia.org/wiki/2017_Abu_Dhabi_G...
23774,23779,988,840,3,18.000,15,18.000,18,18,0.000,...,01:42.3,195.402,11,2017,20,24,Abu Dhabi Grand Prix,2017-11-26,13:00:00,https://en.wikipedia.org/wiki/2017_Abu_Dhabi_G...
23775,23780,988,832,4,55.000,12,NaN,R,19,0.000,...,01:43.4,193.41,36,2017,20,24,Abu Dhabi Grand Prix,2017-11-26,13:00:00,https://en.wikipedia.org/wiki/2017_Abu_Dhabi_G...


Una vez tenga las 3 tablas unidas, empezar con la limpieza de datos